In [39]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

urls = [
    (
        "オカンが好きな朝ごはんを忘れてしまったので、駒場と内海が一緒に忘れてしまった朝ごはんは何かを考える。",
        "https://himapucchi.com/m2019first-milkboy/",
    ),
    (
        "オカンが好きなお菓子を忘れてしまったので、駒場と内海が一緒に忘れてしまったお菓子は何かを考える。",
        "https://himapucchi.com/m2019final-milkboy/",
    ),
    (
        "オカンが好きなイタリアンレストランを忘れてしまったので、駒場と内海が一緒に忘れてしまったイタリアンレストランは何かを考える。",
        "https://himapucchi.com/milkboy-02/",
    ),
]

train_datasets = []

for summary, url in urls:
    # URL先からHTMLを取得
    response = requests.get(url)
    response.encoding = response.apparent_encoding

    # BeautifulSoupでHTML解析
    bs = BeautifulSoup(response.text, "html.parser")
    speach_divs = bs.find_all("div", class_="speech-wrap")

    prompt_count = 0
    speech_history = f"概要:{summary}\n\n###\n\n"
    previous_speaker = "駒場"  # 内海さんから始まるので、前の話者は駒場さんから始める
    speech = ""
    for div in speach_divs:
        if "sbp-r" in div.attrs["class"]:
            speaker = "内海"
        else:
            speaker = "駒場"

        # 話者が前回と同じだったら、前回の発言に追加
        if speaker == previous_speaker:
            speech += " " + div.find("p").get_text()
            continue

        # 発言を更新
        speech = div.find("p").get_text()

        # 内海さんが話者の場合は学習データ格納
        if speaker == "内海":
            prompt_count += 1
            train_datasets.append(
                {"prompt": speech_history + speaker + ":", "completion": speech}
            )

            ## promptが長くなりすぎるので、20回ごとに発言履歴をリセット
            if prompt_count == 20:
                speech_history = f"概要:{summary}\n\n###\n\n"
                prompt_count = 0

        # 発言履歴を更新
        speech_history += f"{speaker}:{speech}\n"

        # 話者を更新
        previous_speaker = speaker

df = pd.DataFrame(train_datasets)
df


,prompt,completion
0,概要:オカンが好きな朝ごはんを忘れてしまったので、駒場と内海が一緒に忘れてしまった朝ごはんは...,どうもー！
1,概要:オカンが好きな朝ごはんを忘れてしまったので、駒場と内海が一緒に忘れてしまった朝ごはんは...,お願いしまーす！
2,概要:オカンが好きな朝ごはんを忘れてしまったので、駒場と内海が一緒に忘れてしまった朝ごはんは...,ありがたいです本当にねー。言うとりますけどもね
3,概要:オカンが好きな朝ごはんを忘れてしまったので、駒場と内海が一緒に忘れてしまった朝ごはんは...,あー、そうなんや
4,概要:オカンが好きな朝ごはんを忘れてしまったので、駒場と内海が一緒に忘れてしまった朝ごはんは...,朝ごはんの名前を忘れてもうて、どうなってんねんそれ
...,...,...
158,概要:オカンが好きなイタリアンレストランを忘れてしまったので、駒場と内海が一緒に忘れてしまっ...,ネズミ講はロイホに行くねんあれは！
159,概要:オカンが好きなイタリアンレストランを忘れてしまったので、駒場と内海が一緒に忘れてしまっ...,ほんまに分からへんやつやんか
160,概要:オカンが好きなイタリアンレストランを忘れてしまったので、駒場と内海が一緒に忘れてしまっ...,ほんやったらオカンの誕生日どこに連れて行くねんな？
161,概要:オカンが好きなイタリアンレストランを忘れてしまったので、駒場と内海が一緒に忘れてしまっ...,いや、絶対ちゃうやろ！もうええわ！


In [40]:
# json形式で保存
df.to_json("dataset.json", orient="records", force_ascii=False)